In [2]:

with open("README.MD", "r", encoding="utf-8") as f:
    summary = f.read()



In [27]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
import gradio as gr
from pydantic import BaseModel, RootModel
load_dotenv(override=True)

True

In [2]:
agents = []

agentsGpt4oMini = {
    "model": "gpt-4o-mini",
    "agent": OpenAI()
}

agents.append(agentsGpt4oMini)

agentsGpt5Mini = {
    "model": "gpt-5-mini",
    "agent": OpenAI()
}

agents.append(agentsGpt5Mini)
google_api_key = os.getenv("GOOGLE_API_KEY")
agentsGemini25Flash = {
    "model": "gemini-2.5-flash",
    "agent": OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/" )
}
agents.append(agentsGemini25Flash)
        
agents

[{'model': 'gpt-4o-mini', 'agent': <openai.OpenAI at 0x1ff48264310>},
 {'model': 'gpt-5-mini', 'agent': <openai.OpenAI at 0x1ff339b34c0>},
 {'model': 'gemini-2.5-flash', 'agent': <openai.OpenAI at 0x1ff48264370>}]

In [3]:
# Carregando recussos 
with open("recursos/every-money-readme.md", "r", encoding="utf-8") as f:
    everyMoneyReadme = f.read()

with open("recursos/criticidade.json", "r", encoding="utf-8") as f:
    criticidade = f.read()


In [40]:
system_prompt = f"""Você é um especialista em análise de riscos e deve analisar o seguinte projeto: {everyMoneyReadme}

Classifique os riscos de acordo com a criticidade: {criticidade}

Apresente pelo menos 3 riscos, classificados conforme a criticidade.

Não seja muito verboso na descrição dos riscos, sendo sucinto e direto.

Deve se atentar para as funcionalidades do projeto, e não apenas as tecnologias utilizadas.

O resultado deve ser apresentado em formato JSON, seguindo o modelo abaixo:
{{
    "Riscos": [
        {{
            "nome": "Nome do risco",
            "criticidade": "Criticidade do risco",
            "descricao": "Descrição do risco",
            "dano_causado": "Dano causado pelo risco",
            "probabilidade_de_ocorrencia": "Probabilidade de ocorrência do risco",
            "impacto": "Impacto do risco"
        }}
    ]
}}"""

In [38]:
from typing import List
class Risco(BaseModel):
    nome: str
    criticidade: str
    descricao: str
    dano_causado: str
    probabilidade_de_ocorrencia: str
    impacto: str

class Riscos(BaseModel):
    riscos: List[Risco]

In [39]:
agentGPt = OpenAI()



response = agentGPt.chat.completions.parse(model= "gpt-5-nano", messages=[{"role": "user", "content": system_prompt}], response_format=Riscos)
answer = response.choices[0].message.parsed
print(answer, ) 

riscos=[Risco(nome='Vazamento/comprometimento de dados sensíveis', criticidade='Alta', descricao='Risco de exposição de dados de usuários, transações e saldos por falha de segurança, JWT mal configurado ou logs expostos.', dano_causado='Exposição de dados confidenciais, violação de privacidade, consequências legais e danos à reputação.', probabilidade_de_ocorrencia='Média', impacto='Alto'), Risco(nome='Inconsistência de saldos e cálculos de orçamento', criticidade='Alta', descricao='Possíveis bugs na lógica de cálculo de saldo realizado/previsto que causem divergência entre transações e saldos.', dano_causado='Saldos incorretos, decisões financeiras equivocadas, insatisfação do usuário.', probabilidade_de_ocorrencia='Média', impacto='Alto'), Risco(nome='Indisponibilidade do sistema / falha de infraestrutura', criticidade='Alta', descricao='Downtime do backend ou falha no PostgreSQL que afete a disponibilidade do sistema.', dano_causado='Interrupção do serviço, perda de produtividade, p

In [20]:
 response.choices[0].message

ParsedChatCompletionMessage[Risco](content='{\n  "nome": "Autenticação e autorização inadequadas",\n  "criticidade": "Alta",\n  "descricao": "Gerenciamento de JWT e permissões sem validação adequada pode permitir acesso não autorizado.",\n  "dano_causado": "Acesso a dados de contas e transações; alterações não autorizadas.",\n  "probabilidade_de_ocorrencia": "Alta",\n  "impacto": "Confidencialidade e integridade dos dados financeiros comprometidas; possível fraude financeira."\n}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None, parsed=Risco(nome='Autenticação e autorização inadequadas', criticidade='Alta', descricao='Gerenciamento de JWT e permissões sem validação adequada pode permitir acesso não autorizado.', dano_causado='Acesso a dados de contas e transações; alterações não autorizadas.', probabilidade_de_ocorrencia='Alta', impacto='Confidencialidade e integridade dos dados financeiros comprometidas; possível fraude financeira.'))

In [9]:
model = agents[0]['model']
print(model)
agent = agents[0]['agent']

response = agent.chat.completions.create(model=model, messages=[{"role": "user", "content": system_prompt}])
answer = response.choices[0].message.content

gpt-4o-mini


In [44]:
print(answer)

```json
{
    "riscos": [
        {
            "Risco": "Inconsistência nos Cálculos de Saldo",
            "criticidade": "Alta",
            "descricao": "Falhas na lógica de cálculo dos saldos (realizado e previsto) das contas, bem como na agregação de transações e orçamentos, resultando em informações financeiras incorretas para o usuário.",
            "Dano Causado": "Informações financeiras erradas levam a decisões equivocadas do usuário, perda de confiança no sistema, e inviabilidade da aplicação para seu propósito central.",
            "Probabilidade de Ocorrência": "Média",
            "Impacto": "Alto"
        },
        {
            "Risco": "Vazamento de Dados Sensíveis ou Acesso Não Autorizado",
            "criticidade": "Alta",
            "descricao": "Vulnerabilidades na implementação de autenticação (JWT), autorização ou na segurança dos endpoints da API e do banco de dados, permitindo acesso não autorizado a dados financeiros e pessoais (senhas, valores, e-mails)

In [ ]:
def 